# PowerShell Output Streams, Transcripts for Logging
Some information about providing history, insights, and artifacts for posterity:  returning reusable objects, and writing informational messages to existing PowerShell output streams

## What
Consumers of our code need understand outcomes, and to know what is the progress / success / failure of the code

## How
PowerShell provides standard ways to provide such outcomes (in the form of rich objects with pertinent properties) and to provide human-readable output streams, as well
- Outcomes: by providing outcomes in object form, we not only inform customer understanding of actions taken, but also enable subsequent value for use cases like reporting, summation, "undo", etc
- Output streams: PowerShell provides for writing human-targeted text items to places like the PowerShell console and eventually to a transcript file for when a consumer desires to capture such "story" info for posterity. Example:  We can employ the following cmdlets for writing human-readable messages to the various output streams: `Write-Verbose`, `Write-Warning`, `Write-Information`

## What this enables (and avoids)
- Standardization: using same patterns / practices that we use in the rest of our PowerShell world leads to success with minimal additional effort (`Start-Transcript` for log file creation, [PowerShell output streams](https://docs.microsoft.com/en-us/powershell/module/microsoft.powershell.core/about/about_output_streams) for information messages like verbose, warning streams)
- Parallelization: by using standard "get/create objects" approaches for outcomes and information streams for human-readable messaging, we are well positioned for when we want/need to explore optimization by employing parallel code execution (by avoiding any filesystem management problems altogether -- things that could become problems like file locks introduced with direct file manipulation)

## Examples
Some examples of emitting objects for further consumption _and_ writing informational / human-readable details to the appropriate PowerShell output streams

The examples illustrate:
- writing code that acts like any other PowerShell cmdlet with which we are all accustomed:  gets objects, does something with objects, emits objects
    - emitting some reusable objects from actions taken, for subsequent use/re-use for other use cases (possibly ones not yet even imagined by the code author)
    - this is represented by the code in the function `Remove-CoolUser.ps1`, which uses a "helper" script that, for the purposes of these examples, introduces success/failure results (`Remove-SomeCoolThing.ps1`)
- writing human-focused "story" information to PowerShell output streams
    - uses an "enhanced" function for prepending Verbose messages with a standardized, parseable timestamp (timestamp can be used to create rich `DateTime` object if ever needed)
    - the "enhanced" function is in `Write-VerboseEnhanced.ps1`
    - also uses the Warning stream (as leveraged in helper simulation script `Remove-SomeCoolThing.ps1`)
- using a "parent" script for:
    - orchestrating the invocation of the user removal
    - leveraging inbuilt, standard "create filesystem log artifact" mechanism for logging the human-focused story bits (via `Start-Transcript`)
    - and, serializing ("stringifying") rich objects for later reconsumption (converting to JSON, writing to file, emitting a corresponding `System.IO.FileInfo` PowerShell object)
    - this is represented by the code in the function `Invoke-CoolUserRemoval.ps1`
- follows well established usage patterns in PowerShell by which consumer can control verbosity level (via `-Verbose` parameter to main orchestration code)

#### Writing code that acts like any other PowerShell cmdlet

In [5]:
.\OutputStreamsAndLogging\Remove-CoolUser.ps1 -User dickie, roscoe


User   Guid                                 RemovedSuccessfully RemovalDatetime
----   ----                                 ------------------- ---------------
dickie d84b8828-2e6f-4f0d-a964-dfecc1134885                True 7/11/2022 1:37:41 PM
roscoe 2ec67b7d-1a60-407b-8110-2dbc21aad5c3                True 7/11/2022 1:37:41 PM



#### Writing human-focused "story" information to PowerShell output streams
By leveraging the familiar `-Verbose` parameter, our information output streams now have the human-focused story, too (but, using the same code!)

In [19]:
.\OutputStreamsAndLogging\Remove-CoolUser.ps1 -User dickie, roscoe -Verbose

VERBOSE: 2022-07-11T14:03:35.7630028-04:00 [Remove-CoolUser.ps1] Starting all the goodness
VERBOSE: 2022-07-11T14:03:35.7762737-04:00 [Remove-CoolUser.ps1] Working on item 'dickie'
VERBOSE: 2022-07-11T14:03:35.7886026-04:00 [Remove-CoolUser.ps1] Deleting user 'dickie' from some system
VERBOSE: 2022-07-11T14:03:35.8237171-04:00 [Remove-SomeCoolThing.ps1] Removing cool thing 'dickie'

VERBOSE: 2022-07-11T14:03:36.0920034-04:00 [Remove-CoolUser.ps1] Working on item 'roscoe'
VERBOSE: 2022-07-11T14:03:36.1047535-04:00 [Remove-CoolUser.ps1] Deleting user 'roscoe' from some system
VERBOSE: 2022-07-11T14:03:36.1333352-04:00 [Remove-SomeCoolThing.ps1] Removing cool thing 'roscoe'
User   Guid                                 RemovedSuccessfully RemovalDatetime
----   ----                                 ------------------- ---------------
dickie f03c6892-a01f-42a8-a9af-2a9a725587db                True 7/11/2022 2:03:36 PM
roscoe 3da06c03-0320-4bcc-9649-c0c3c41c570b                True 7/11/2022 2

#### Using a "parent" script for non-interactive invocation

In [20]:
.\OutputStreamsAndLogging\Invoke-CoolUserRemoval.ps1 -Verbose -OutVariable oOutputFileinfo

VERBOSE: Transcript started, output file is C:\Users\sweetPuter0\Documents\PowerShell_transcript.sweetPuter0.1bSIhY6E.20220711140652.txt
VERBOSE: 2022-07-11T14:06:52.9128816-04:00 [Remove-CoolUser.ps1] Starting all the goodness
VERBOSE: 2022-07-11T14:06:52.9390162-04:00 [Remove-CoolUser.ps1] Working on item 'user1608'
VERBOSE: 2022-07-11T14:06:52.9539286-04:00 [Remove-CoolUser.ps1] Deleting user 'user1608' from some system
VERBOSE: 2022-07-11T14:06:52.9845775-04:00 [Remove-SomeCoolThing.ps1] Removing cool thing 'user1608'
VERBOSE: 2022-07-11T14:06:53.2535155-04:00 [Remove-CoolUser.ps1] Working on item 'user1899'
VERBOSE: 2022-07-11T14:06:53.2666033-04:00 [Remove-CoolUser.ps1] Deleting user 'user1899' from some system
VERBOSE: 2022-07-11T14:06:53.2987742-04:00 [Remove-SomeCoolThing.ps1] Removing cool thing 'user1899'
VERBOSE: 2022-07-11T14:06:53.5628046-04:00 [Remove-CoolUser.ps1] Working on item 'user1657'
VERBOSE: 2022-07-11T14:06:53.5778149-04:00 [Remove-CoolUser.ps1] Deleting user '

Here we reused the "enhanced" verbose function, leveraged the Warning output stream as well, and emitted the `FileInfo` object that shows the details of the JSON file in which the rich objects were serialized

Then, we can use the FileInfo object (or just the full path to the emitted, serialized objects file) to subsequently do interesting stuff:

In [21]:
## see the file
Get-ChildItem $oOutputFileinfo


    Directory: C:\Users\sweetPuter0\AppData\Local\Temp

Mode                 LastWriteTime         Length Name
----                 -------------         ------ ----
-a---           7/11/2022  2:06 PM           1833 deletedUsers-20220711T1406528848.json



In [22]:
## "rehydrate" (deserialize) the objects
Get-ChildItem $oOutputFileinfo | Get-Content | ConvertFrom-Json


User     Guid                                 RemovedSuccessfully RemovalDatetime
----     ----                                 ------------------- ---------------
user1608 e2cc5245-2f51-4c39-8ff8-6c0453e73f80                True 7/11/2022 2:06:53 PM
user1899 ed6e0ee7-a230-4072-83b8-6dc1fa3752f0                True 7/11/2022 2:06:53 PM
user1657 e77d39e2-2c49-4a93-96e5-4ed12a1535af                True 7/11/2022 2:06:53 PM
user1359 a9941940-8b0c-4e5d-93d4-7486b669b0f5               False 7/11/2022 2:06:54 PM
user1250 987d8085-b5fa-4905-a54a-3a1df0f08531                True 7/11/2022 2:06:54 PM
user1184 de0d7e0d-864d-4d55-a6fd-eac9a7915309                True 7/11/2022 2:06:54 PM
user1982 89d2a4b9-aba8-4c3e-ae78-ac62c7736282               False 7/11/2022 2:06:55 PM
user1142 c83e4086-2486-4adc-9ea0-8de20cd3d3ea                True 7/11/2022 2:06:55 PM
user1299 cf9ab935-1cea-425c-aa99-e960bfec408e                True 7/11/2022 2:06:55 PM
user1552 ffb60438-bdd6-403c-a8de-bd5477f187a0       

In [26]:
## do some filtering on the once again rich objects
Get-ChildItem $oOutputFileinfo | Get-Content | ConvertFrom-Json | Where-Object -not RemovedSuccessfully


User     Guid                                 RemovedSuccessfully RemovalDatetime
----     ----                                 ------------------- ---------------
user1359 a9941940-8b0c-4e5d-93d4-7486b669b0f5               False 7/11/2022 2:06:54 PM
user1982 89d2a4b9-aba8-4c3e-ae78-ac62c7736282               False 7/11/2022 2:06:55 PM

